## Paralelización

In [1]:
import random
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from matplotlib import cm
import scipy.optimize as opt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import minimize
from joblib import Parallel, delayed 
import matplotlib.colorbar as colorbar
from matplotlib.colors import Normalize
from IPython.display import display, Markdown
from matplotlib.colors import LinearSegmentedColormap

### Simulación de portafolios

In [4]:
tickers = ['AAPL', 'NVDA', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'SPOT', 'SBUX', 'JPM', 'AXP', 'MCD', 'KO', 
           'NFLX', 'CMG', 'CP', 'WMT', 'V', 'GLD', 'BLK', 'PG', 'JNJ', 'TMUS', 'MA', 'BX', 'LULU', 'DPZ', 'BAC', 
           'FDX', 'DIS', 'GE', 'HSY', 'HP', 'COST', 'HD', 'K', 'ADBE', 'CSCO', 'T', 'F', 'NKE', 'CVX', 'XOM', 'PYPL', 
           'PEP', 'PFE', 'MRNA', 'RL', 'AZN', 'BABA', 'VZ', 'WBD', 'HSBC', 'UBER']

rf = 0.04413
f_inicial='2020-01-01'
f_final='2024-11-24'

In [5]:
def simulate_portfolio_opt(i, tickers, rf):
    # Selección aleatoria del tamaño de la muestra y de los tickers
    sample_size = random.randint(5, 15)
    random_tickers = random.sample(tickers, sample_size)

    # Descarga de datos
    datos1 = yf.download(random_tickers, f_inicial, f_final)['Adj Close']
    rt = datos1.pct_change().dropna()

    # Calcular retornos y matriz de covarianza
    mu = (rt.mean() * 252).values  # retornos esperados
    sigma = rt.cov().values  # Matriz de covarianza
    n_assets = len(mu)

    # Función para minimizar (-Sharpe Ratio)
    def neg_sharpe_ratio(w, mu, sigma, rf):
        port_return = np.dot(w, mu)  # Rendimiento esperado del portafolio
        port_vol = np.sqrt(np.dot(w.T, np.dot(sigma, w))) * np.sqrt(252)  # Volatilidad del portafolio
        sharpe_ratio = (port_return - rf) / port_vol
        return -sharpe_ratio  # Negativo porque queremos maximizar

    # Restricciones: Suma de pesos = 1
    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })

    # Límites: Pesos entre 0 y 1 (no posiciones cortas)
    bounds = tuple((0, 1) for _ in range(n_assets))

    # Pesos iniciales (distribuidos uniformemente)
    w0 = np.array([1 / n_assets] * n_assets)

    # Optimización
    result = opt.minimize(neg_sharpe_ratio, w0, args=(mu, sigma, rf), method='SLSQP', bounds=bounds, constraints=constraints)

    # Resultados
    optimal_weights = result.x
    optimal_weights[optimal_weights < 0.000001] = 0  # Eliminar pesos numéricamente cercanos a cero

    # Cálculo del retorno, volatilidad y sharpe del portafolio
    ret = np.dot(optimal_weights.T, mu)  # Retorno esperado del portafolio
    risk = np.sqrt(np.dot(optimal_weights.T, np.dot(sigma, optimal_weights))) * np.sqrt(252)  # Volatilidad del portafolio
    sharpe = (ret - rf) / risk  # Sharpe ratio

    return ret, risk, sharpe, optimal_weights

# Número de simulaciones
n_simulations = 100

# Ejecutar simulaciones en paralelo
results = Parallel(n_jobs=-1)(delayed(simulate_portfolio_opt)(i, tickers, rf) for i in range(n_simulations))

# Extraer resultados
retornos_sci = [result[0] for result in results]
volatilidades_sci = [result[1] for result in results]
sharpe_ratios_sci = [result[2] for result in results]
# pesos_sci = [result[3] for result in results]

[*********************100%***********************]  7 of 7 completedd
[*********************100%***********************]  8 of 8 completed
[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  8 of 8 completeded
[*********************100%***********************]  8 of 8 completeded
[*********************100%***********************]  13 of 13 completed
[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  7 of 7 completedd
[*********************100%***********************]  7 of 7 completedd
[*********************100%***********************]  9 of 9 completeddd
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  14 of 14 completed
[************